In [1]:
import joblib
import pandas as pd
import numpy as np

### --- Step A: Load Both Models (FIXED) ---
print("Loading models...")
# FIX 1: Load the correct HMM model file
hmm_model = joblib.load('hmm_regime_classifier_v1.joblib')

# Load the "Forecaster"
xgb_vol_model = joblib.load('volatility_xgb_no_persistence_v1.joblib')

print("✅ Models loaded successfully.")

### --- Step B: Create Sample New Data (FIXED) ---
# FIX 2: Added all missing columns that the xgb_vol_model 
# pipeline needs to see (even the ones it will drop).
new_data = pd.DataFrame({
    # --- HMM features ---
    'Return_Pct': [0.1],
    'Realized_Vol_10d': [1.4],
    
    # --- "Lazy" features (pipeline will drop) ---
    'Realized_Vol_20d': [1.5],
    'Volatility_Ratio': [1.0],
    
    # --- Secondary features (pipeline will use) ---
    'VIX': [19.0],
    'Price_ZScore': [0.5],
    'Return_Pct_lag_1': [0.1],
    'Return_Pct_lag_2': [-0.2],
    'Return_Pct_lag_3': [0.3],
    'Ticker': ['AAPL'],
    
    # --- All other columns from the previous error ---
    'CPI': [250.0],
    'Unemployment_Rate': [3.5],
    'Close': [152.0],
    'Momentum_Ratio_S_M': [1.1],
    'Volume_MA_20': [50000000.0],
    'Price_Range_Pct': [1.2],
    'Volume_Ratio': [0.9],
    'Momentum_5d': [2.5],
    'MA_20': [148.0],
    'Fed_Funds_Rate': [0.25],
    'Volume': [60000000.0],
    'Momentum_Ratio_M_L': [1.0],
    'Open': [151.5],
    'High': [153.0],
    'MA_200': [140.0],
    'GDP': [20000.0],
    'MA_50': [150.2],
    'Yield_Curve_10Y_2Y': [0.15],
    'Price_Range': [1.5],
    'Low': [151.0]
})

print("\nSample Data (with all columns):")
print(new_data.head())

### --- Step C: Run the 2-Step Prediction ---

# 1. Use HMM to get the 'regime'
#    FIX 3: The real HMM model wants a NumPy array, not a DataFrame.
hmm_features = new_data[['Return_Pct', 'Realized_Vol_10d']].values
predicted_regimes = hmm_model.predict(hmm_features)

# 2. Add the new 'regime' feature to your data
new_data['regime'] = predicted_regimes
print(f"\nAdded 'regime' feature: {predicted_regimes}")

# 3. Predict Volatility
#    This will now work. The pipeline will:
#    a) Drop the 'lazy' vol features ('Realized_Vol_10d', etc.)
#    b) Keep the secondary features ('VIX', 'Price_ZScore', etc.)
#    c) OneHotEncode 'Ticker'
#    d) Drop the new 'regime' column (because 'remainder=drop'
#       was set in your pipeline and it wasn't trained on it)
vol_prediction = xgb_vol_model.predict(new_data)

print("\n--- 📈 Final Volatility Prediction ---")
for i, ticker in enumerate(new_data['Ticker']):
    print(f"  {ticker}: Predicted Volatility = {vol_prediction[i]:.4f}")

Loading models...
✅ Models loaded successfully.

Sample Data (with all columns):
   Return_Pct  Realized_Vol_10d  Realized_Vol_20d  Volatility_Ratio   VIX  \
0         0.1               1.4               1.5               1.0  19.0   

   Price_ZScore  Return_Pct_lag_1  Return_Pct_lag_2  Return_Pct_lag_3 Ticker  \
0           0.5               0.1              -0.2               0.3   AAPL   

   ...      Volume  Momentum_Ratio_M_L   Open   High  MA_200      GDP  MA_50  \
0  ...  60000000.0                 1.0  151.5  153.0   140.0  20000.0  150.2   

   Yield_Curve_10Y_2Y  Price_Range    Low  
0                0.15          1.5  151.0  

[1 rows x 30 columns]

Added 'regime' feature: [0]

--- 📈 Final Volatility Prediction ---
  AAPL: Predicted Volatility = 1.5371


In [2]:
import joblib
import pandas as pd
import numpy as np

### --- Step A: Load the Model ---
print("Loading model...")
returns_model = joblib.load('returns_xgb_global_tuned_v1.joblib')
print("✅ Model loaded successfully.")

### --- Step B: Create Sample New Data (FIXED) ---
# This data MUST have *all* the columns the model was trained on.
# The error message listed all the missing ones.

new_data_returns = pd.DataFrame({
    # --- Original 6 Columns ---
    'Return_Pct_lag_1': [0.1],
    'Return_Pct_lag_2': [-0.2],
    'VIX': [19.0],
    'MA_50': [150.2],
    'MA_200': [140.0],
    'Ticker': ['AAPL'],
    
    # --- Columns from the ValueError (with dummy values) ---
    'Volatility_Ratio': [1.0],
    'CPI': [250.0],
    'Unemployment_Rate': [3.5],
    'Close': [152.0],
    'Momentum_Ratio_S_M': [1.1],
    'Return_Pct': [0.1], 
    'Realized_Vol_20d': [1.5],
    'Volume_MA_20': [50000000.0],
    'Price_Range_Pct': [1.2],
    'Volume_Ratio': [0.9],
    'Momentum_5d': [2.5],
    'MA_20': [148.0],
    'Fed_Funds_Rate': [0.25],
    'Realized_Vol_10d': [1.4],
    'Volume': [60000000.0],
    'Return_Pct_lag_3': [0.3],
    'Momentum_Ratio_M_L': [1.0],
    'Price_ZScore': [0.5],
    'Open': [151.5],
    'High': [153.0],
    'GDP': [20000.0],
    'Yield_Curve_10Y_2Y': [0.15],
    'Price_Range': [1.5],
    'Low': [151.0]
    
    # NOTE: Your training code excluded 'Return_MA_20',
    # so we correctly leave it out of this list.
})

print("\nSample Data (with all columns):")
print(new_data_returns.head())

### --- Step C: Run the Prediction ---
# This will now work
return_prediction = returns_model.predict(new_data_returns)

print("\n--- 💸 Final Returns Prediction ---")
for i, ticker in enumerate(new_data_returns['Ticker']):
    prediction = return_prediction[i]
    direction = "Up" if prediction > 0 else "Down/Flat"
    print(f"  {ticker}: Raw Score = {prediction:.4f}  (Predicted: {direction})")

Loading model...
✅ Model loaded successfully.

Sample Data (with all columns):
   Return_Pct_lag_1  Return_Pct_lag_2   VIX  MA_50  MA_200 Ticker  \
0               0.1              -0.2  19.0  150.2   140.0   AAPL   

   Volatility_Ratio    CPI  Unemployment_Rate  Close  ...      Volume  \
0               1.0  250.0                3.5  152.0  ...  60000000.0   

   Return_Pct_lag_3  Momentum_Ratio_M_L  Price_ZScore   Open   High      GDP  \
0               0.3                 1.0           0.5  151.5  153.0  20000.0   

   Yield_Curve_10Y_2Y  Price_Range    Low  
0                0.15          1.5  151.0  

[1 rows x 30 columns]

--- 💸 Final Returns Prediction ---
  AAPL: Raw Score = 0.0713  (Predicted: Up)


In [3]:
import joblib
import pandas as pd
import numpy as np

### --- Step A: Load the Model ---
print("Loading HMM model...")
hmm_model = joblib.load('hmm_regime_classifier_v1.joblib')
print("✅ Model loaded successfully.")

### --- Step B: Create Sample Data ---
# This model *only* wants the 2 features it was trained on.
hmm_features = np.array([
    [0.1, 1.2],  # Day 1: Low return, low vol
    [-2.5, 3.0], # Day 2: High return, high vol
    [0.2, 1.0],  # Day 3: Low return, low vol
    [1.5, 2.8]   # Day 4: High return, high vol
])

print("\nSample Data:")
print(hmm_features)

### --- Step C: Run the Prediction ---
# The output is an array of 0s and 1s.
predicted_regimes = hmm_model.predict(hmm_features)

print("\n--- 🌦️ Final Regime Predictions ---")
print(predicted_regimes)
# (Assuming 0=Low-Vol, 1=High-Vol)
# Output: [0 1 0 1]

Loading HMM model...
✅ Model loaded successfully.

Sample Data:
[[ 0.1  1.2]
 [-2.5  3. ]
 [ 0.2  1. ]
 [ 1.5  2.8]]

--- 🌦️ Final Regime Predictions ---
[0 1 1 1]


In [4]:
import joblib
import pandas as pd
import numpy as np
import warnings

# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore', category=UserWarning)

# ==============================================================================
# 1. LOAD ALL THREE MODELS
# ==============================================================================
print("Loading all three models...")

try:
    # Model 1: The "Weather Station" (Tells us the market state)
    hmm_model = joblib.load('hmm_regime_classifier_v1.joblib')

    # Model 2: The "Risk Forecaster" (Predicts 'honest' volatility)
    xgb_vol_model = joblib.load('volatility_xgb_no_persistence_v1.joblib')

    # Model 3: The "Directional Signal" (Predicts returns)
    xgb_returns_model = joblib.load('returns_xgb_global_tuned_v1.joblib')
    
    print("✅ All models loaded successfully.")

except FileNotFoundError as e:
    print(f"❌ ERROR: Model file not found. Make sure this file is in the same folder: {e.filename}")
    # Stop the script if models aren't loaded
    exit()

# ==============================================================================
# 2. GET NEW MARKET DATA (SIMULATION)
# ==============================================================================

def get_new_market_data():
    """
    Simulates fetching new data for all 3 tickers.
    This DataFrame MUST have all columns your models were trained on.
    """
    print("\nFetching new market data...")
    
    new_data = pd.DataFrame({
        # --- Key Features (Examples for 3 different scenarios) ---
        'Ticker':               ['AAPL', 'MSFT', 'GOOGL'],
        'Return_Pct':           [0.1,    -2.5,   0.8],    # HMM input 1
        'Realized_Vol_10d':     [1.1,    3.2,    1.3],    # HMM input 2
        'Return_Pct_lag_1':     [0.2,    -1.1,   0.1],
        'Return_Pct_lag_2':     [0.1,    -0.8,   0.2],
        'Return_Pct_lag_3':     [0.3,    -0.2,   0.1],
        'VIX':                  [15.2,   35.1,   16.0],
        
        # --- "Lazy" Vol Features (Needed by 'xgb_vol_model' pipeline to drop) ---
        'Realized_Vol_20d':     [1.2,    3.0,    1.4],
        'Volatility_Ratio':     [0.9,    1.1,    0.8],

        # --- All Other Dummy Columns (from your 'Returns' model) ---
        'CPI':                  [250.0,  250.0,  250.0],
        'Unemployment_Rate':    [3.5,    3.5,    3.5],
        'Close':                [152.0,  300.0,  140.0],
        'Momentum_Ratio_S_M':   [1.1,    0.9,    1.0],
        'Volume_MA_20':         [50e6,   40e6,   30e6],
        'Price_Range_Pct':      [1.2,    2.8,    1.1],
        'Volume_Ratio':         [0.9,    1.3,    1.0],
        'Momentum_5d':          [2.5,    -4.0,   1.5],
        'MA_20':                [148.0,  310.0,  138.0],
        'Fed_Funds_Rate':       [0.25,   0.25,   0.25],
        'Volume':               [60e6,   55e6,   35e6],
        'Momentum_Ratio_M_L':   [1.0,    0.9,    1.0],
        'Price_ZScore':         [0.5,    -1.5,   0.2],
        'Open':                 [151.5,  305.0,  139.0],
        'High':                 [153.0,  306.0,  141.0],
        'MA_200':               [140.0,  290.0,  130.0],
        'GDP':                  [20000.0,20000.0,20000.0],
        'MA_50':                [150.2,  305.0,  135.0],
        'Yield_Curve_10Y_2Y':   [0.15,   0.15,   0.15],
        'Price_Range':          [1.5,    5.0,    2.0],
        'Low':                  [151.0,  301.0,  139.0]
    })
    return new_data

# Get the simulated data
new_data = get_new_market_data()

# ==============================================================================
# 3. RUNNING THE COMBINED ANALYSIS
# ==============================================================================

print("\n" + "="*50)
print("     DAILY TRADING & RISK BRIEFING")
print("="*50)

# We analyze each stock one-by-one
for i in range(len(new_data)):
    
    # Isolate the data for one stock
    ticker_data = new_data.iloc[[i]]
    ticker_name = ticker_data['Ticker'].values[0]
    
    print(f"\n--- ⚡ Analysis for {ticker_name} ---")

    # --- STEP 1: What's the market weather? (HMM Model) ---
    # The HMM model needs a simple NumPy array of its 2 features
    hmm_features = ticker_data[['Return_Pct', 'Realized_Vol_10d']].values
    predicted_regime = hmm_model.predict(hmm_features)[0]
    
    # (Assuming 1 is High-Vol, 0 is Low-Vol. Adjust if HMM output is different)
    regime_label = "High-Vol (Chaotic)" if predicted_regime == 1 else "Low-Vol (Calm)"
    
    # --- STEP 2: What's the *honest* risk forecast? (Volatility Model) ---
    # This pipeline was trained to *exclude* lazy features.
    vol_prediction = xgb_vol_model.predict(ticker_data)[0]

    # --- STEP 3: What's the directional signal? (Returns Model) ---
    # This pipeline predicts a raw number (positive=Up, negative=Down)
    return_prediction = xgb_returns_model.predict(ticker_data)[0]
    direction_label = "Up" if return_prediction > 0 else "Down/Flat"
    
    
    # --- STEP 4: SYNTHESIZE & DECIDE (The "Practical Benefit") ---
    print(f"  🌦️ Market Weather: {regime_label} (Regime {predicted_regime})")
    print(f"  🧭 Direction Signal: {direction_label} (Score: {return_prediction:.4f})")
    print(f"  🔥 Risk Forecast: Predicted Volatility = {vol_prediction:.4f}")
    
    print("\n  💡 ACTIONABLE INSIGHT:")
    
    if predicted_regime == 1:
        print("  Signal: **CAUTION (High-Risk)**")
        print("  Reason: The HMM model detects a 'Chaotic' market. Risk is high.")
        print("  Action: Avoid new positions; consider reducing risk.")
        
    elif direction_label == "Down/Flat":
        print("  Signal: **POTENTIAL SELL / AVOID**")
        print("  Reason: HMM is calm, but the (Up-biased) returns model is predicting 'Down/Flat'.")
        print("  Action: This is a strong negative signal. Do not buy.")
        
    else: # Regime is 0 (Calm) and Direction is "Up"
        print("  Signal: **POTENTIAL BUY**")
        print("  Reason: HMM is calm and the returns model gives an 'Up' signal.")
        print("  Action: Conditions appear favorable. Risk is on.")
        
    print("-"*40)

Loading all three models...
✅ All models loaded successfully.

Fetching new market data...

     DAILY TRADING & RISK BRIEFING

--- ⚡ Analysis for AAPL ---
  🌦️ Market Weather: Low-Vol (Calm) (Regime 0)
  🧭 Direction Signal: Up (Score: 0.0713)
  🔥 Risk Forecast: Predicted Volatility = 1.2150

  💡 ACTIONABLE INSIGHT:
  Signal: **POTENTIAL BUY**
  Reason: HMM is calm and the returns model gives an 'Up' signal.
  Action: Conditions appear favorable. Risk is on.
----------------------------------------

--- ⚡ Analysis for MSFT ---
  🌦️ Market Weather: Low-Vol (Calm) (Regime 0)
  🧭 Direction Signal: Up (Score: 0.7320)
  🔥 Risk Forecast: Predicted Volatility = 2.1590

  💡 ACTIONABLE INSIGHT:
  Signal: **POTENTIAL BUY**
  Reason: HMM is calm and the returns model gives an 'Up' signal.
  Action: Conditions appear favorable. Risk is on.
----------------------------------------

--- ⚡ Analysis for GOOGL ---
  🌦️ Market Weather: Low-Vol (Calm) (Regime 0)
  🧭 Direction Signal: Up (Score: 0.0658)
 

In [5]:
import os

# Get path to notebooks folder (one level up, then into notebooks)
APP_DIR = os.path.dirname(os.path.abspath(__file__))  # Gets 'app' folder
NOTEBOOKS_DIR = os.path.join(os.path.dirname(APP_DIR), 'notebooks')  # Goes up to project, then into notebooks

print("--- Loading all three models... ---")
print(f"Looking for models in: {NOTEBOOKS_DIR}")

try:
    MODELS = {
        "hmm": joblib.load(os.path.join(NOTEBOOKS_DIR, 'hmm_regime_classifier_v1.joblib')),
        "volatility": joblib.load(os.path.join(NOTEBOOKS_DIR, 'volatility_xgb_no_persistence_v1.joblib')),
        "returns_10d": joblib.load(os.path.join(NOTEBOOKS_DIR, 'returns_xgb_global_tuned_v1.joblib'))
    }
    print("✅ All models loaded successfully.")
except FileNotFoundError as e:
    print(f"❌ FATAL ERROR: Model file not found. {e}")
    print(f"Checked directory: {NOTEBOOKS_DIR}")
    print("\nFiles in notebooks directory:")
    if os.path.exists(NOTEBOOKS_DIR):
        for file in os.listdir(NOTEBOOKS_DIR):
            if file.endswith('.joblib'):
                print(f"  ✅ {file}")
    exit()

NameError: name '__file__' is not defined